In [0]:
import os, sys
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.patches import Ellipse, Circle
from matplotlib.gridspec import GridSpec
from scipy.special import logsumexp
from scipy.interpolate import interp1d

from astropy import wcs
from astropy import units as u
from astropy.coordinates import Angle, SkyCoord
from astropy.time import Time

In [0]:
plt.rc('font', size=6) #controls default text size

In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
from pfs.ga.targeting.targets.dsph import *
from pfs.ga.targeting.instrument import *
from pfs.ga.targeting.diagram import CMD, CCD, FOV, FP, ColorAxis, MagnitudeAxis
from pfs.ga.targeting.photometry import Photometry, Magnitude, Color
from pfs.ga.targeting.projection import WcsProjection, Pointing
from pfs.ga.targeting.netflow import Netflow

In [0]:
pointing = Pointing(0, 0, posang=0)
wcs = WcsProjection(pointing, proj='TAN')
wfc = SubaruWFC(pointing)
fov = FOV(projection=wcs)
fp = FP(wfc)

In [0]:
f, ax = plt.subplots(1, 1, figsize=(8, 8), dpi=240)

pointing = Pointing(0, 0, posang=0)
pfi = SubaruPFI(instrument_options={ 'layout': 'calibration' })

pfi.plot_focal_plane(ax, fp, corners=True, projection=SubaruWFC(pointing))
pfi.plot_cobras(ax, fp)
# pfi.plot_fiber_numbers(ax, fp, fontsize=2.5)

In [0]:
cmap = plt.get_cmap('flag')

# Get the focal plane coordinates of the cobras
ncobras = len(pfi.bench.cobras.centers)
x, y = pfi.bench.cobras.centers[:].real, pfi.bench.cobras.centers[:].imag

# Convert to polar coordinates around the center of the focal plane
r = np.sqrt(x**2 + y**2)
theta = np.arctan2(y, x)

# Assign labels to the cobras based on the polar coordinate
# Use 6 sector in theta and four bins in r
ntheta = 6
theta_bins = np.linspace(-np.pi, np.pi, ntheta + 1)
r_bins = np.array([0, 150, 240])

theta_labels = np.digitize(theta, theta_bins, right=False) - 1
r_labels = np.digitize(r, r_bins, right=False) - 1
cobra_location_labels = (r_bins.size - 1) * theta_labels + r_labels

# Add one more label in the center
cobra_location_labels[r < 60] = cobra_location_labels.max() + 1

f, ax = plt.subplots(1, 1, figsize=(4, 4), dpi=240)

ax.scatter(x, y, s=1, c=cobra_location_labels, cmap=cmap)
ax.set_aspect('equal')

In [0]:
np.bincount(r_labels), np.bincount(theta_labels)

In [0]:
np.bincount(labels), labels.max()

# Cobra locations in the detector

In [0]:
# 0-based indices of cobras in each spectrograph
# 65534 means engineering fiber
pfi.fiber_map.cobrasForSpectrograph(1)

In [0]:
pfi.fiber_map.cobrasForSpectrograph(1).size

In [0]:
cobra_instrument_labels = np.zeros(ncobras, dtype=int)

# For each spectrograph, get the corresponding cobra
for s in np.arange(4) + 1:
    cobra_ids = pfi.fiber_map.cobrasForSpectrograph(s)
    mask = cobra_ids != 65534

    print(s, cobra_ids.shape, mask.sum())

    cobra_instrument_labels[cobra_ids[mask]] = s

f, ax = plt.subplots(1, 1, figsize=(4, 4), dpi=240)

ax.scatter(x, y, s=1, c=cobra_instrument_labels, cmap='tab10')
ax.set_aspect('equal')

In [0]:
from pfs.utils.fibers import fiberHoleFromFiberId

In [0]:
pfi.bench.fiberHoleFromFiberId(0)

In [0]:
pfi.fiber_map.cobrasForSpectrograph(1)

In [0]:
pfi.fiber_map.cobraId, pfi.fiber_map.fiberHoleId

In [0]:
650 / 50

In [0]:
for s in np.arange(4) + 1:
    holeids = pfi.fiber_map.fiberHoleId[pfi.fiber_map.spectrographId == s]
    print(s, holeids.min(), holeids.max())
    


In [0]:
cobra_instrument_labels = np.zeros(ncobras, dtype=int)
mask = pfi.fiber_map.cobraId != 65535

# cobra_instrument_labels[pfi.fiber_map.cobraId[mask] - 1] = pfi.fiber_map.spectrographId[mask]

ngroups = 8
fiber_hole_group_size = pfi.fiber_map.fiberHoleId.max() / ngroups
print(fiber_hole_group_size)
cobra_instrument_labels[pfi.fiber_map.cobraId[mask] - 1] = \
    (pfi.fiber_map.spectrographId[mask] - 1) * ngroups \
    + (np.round(pfi.fiber_map.fiberHoleId[mask] - 1) / fiber_hole_group_size).astype(int)
print(cobra_instrument_labels, cobra_instrument_labels.min(), cobra_instrument_labels.max())

f, ax = plt.subplots(1, 1, figsize=(4, 4), dpi=240)

ax.scatter(x, y, s=1, c=cobra_instrument_labels, cmap='tab10')
ax.set_aspect('equal')